## Detectar formacion de canal Bajista - Alcista

##### Configuracion y descarga de datos

In [1]:
import pandas as pd
import numpy as np
#import plotly.graph_objects as go
#import yfinance as yf
import talib as ta
import ta as ta2
#import matplotlib.pyplot as plt
#import math
from datetime import date, timedelta
from scipy import stats
from scipy.signal import argrelextrema
import bokeh as bokeh
from bokeh.plotting import figure, show, column
from bokeh.models import NumeralTickFormatter, Span, LabelSet, ColumnDataSource
from sklearn.linear_model import LinearRegression
import os
from backtesting import Backtest, Strategy

c:\Users\carlo\anaconda3\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
df_h = pd.read_csv("D:/TraderEstrategias/data/dataxh.txt", sep="\t")
tickers = [
    'SPY',
    'AAPL'
]
# tickers = [
#     'SPY',
#     'META',
#     'AAPL',
#     'AMZN',
#     'NFLX',
#     'MRNA',
#     'TSLA',
#     'TNA',
#     'GLD',
#     'SLV',
#     'USO',
#     'BAC',
#     'CVX',
#     'XOM',
#     'QQQ',
#     'MSFT',
#     'NVDA',
#     'WMT',
#     'BA',
#     'DIS',
#     'CAT',
#     'IBM',
#     'WFC',
#     'PLTR',
#     'AMD',
#     'AVGO',
#     'HOOD',
#     'CRWV',
#     'MSTR',
#     'UNH',
#     'GOOG',
#     'APP',
#     'UBER'
# ]

In [3]:
def df_limpiar(df_eval):
    meandif1 = df_eval['ATR'].mean()
    stddif1 = df_eval['ATR'].std()
    topdif1 = meandif1 + stddif1 * 1.96
    copydf = df_eval.copy()
    copydf['ind1'] = copydf.apply(lambda row: 1 if row['ATR'] > topdif1 else 0, axis=1)
    q3_dif1 = copydf[copydf['ind1']!=1]['ATR'].quantile(0.75)

    copydf =copydf.apply(
        lambda row: q3_dif1 if (row['ind1']==1) else 
        row['ATR'], axis=1)
    return copydf

In [4]:
df_h['Datetime'] = pd.to_datetime(df_h['Datetime'])
df = pd.DataFrame()
for ticker in tickers:
    company = df_h.query("companyName==@ticker").copy()
    company.sort_values(by=['Datetime'])
    #company = yf.download(ticker, start = ini2_str, end = today_str, interval='60m')
    #company.columns = [company.columns[0][0], company.columns[1][0], company.columns[2][0], company.columns[3][0], company.columns[4][0]]
    #company.rename(columns={'Datetime':'Gmt time'}, inplace = True)
    #company['Datetime'] = pd.to_datetime (company.index)
    #company['companyName'] = ticker
    #company['SMA35'] = company['Close'].rolling(35).mean()
    #company.dropna(inplace=False)
    #company['SMA50'] = company['Close'].rolling(50).mean()
    #company.dropna(inplace=False)
    company['EMA35'] = company['Close'].ewm(span=35, adjust=False).mean()
    company.dropna(inplace=False)
    company['EMA50'] = company['Close'].ewm(span=50, adjust=False).mean()
    company.dropna(inplace=False)

    company['EMA20'] = company['Close'].ewm(span=20, adjust=False).mean()
    company.dropna(inplace=False)
    company['EMA10'] = company['Close'].ewm(span=10, adjust=False).mean()
    company.dropna(inplace=False)
    company['EMA5'] = company['Close'].ewm(span=5, adjust=False).mean()
    company.dropna(inplace=False)

    #company['EMA40'] = company['Close'].ewm(span=40, adjust=False).mean()
    #company.dropna(inplace=False)

    # Calcular +DI, -DI y ADX (todo el DMI)
    dmi = ta2.trend.ADXIndicator(high=company['High'], low=company['Low'], close=company['Close'], window=14)
    company['plus_di'] = dmi.adx_pos()   # +DI
    company['minus_di'] = dmi.adx_neg()  # -DI
    company['adx'] = dmi.adx()           # ADX

    #ATR indicador para Trailing Stop Loss
    company['ATR'] = ta2.volatility.average_true_range(company['High'], company['Low'], company['Close'], window=14)
    company['ATR2'] = df_limpiar(company)
    company['EMA35_ATR'] = company['ATR2'].ewm(span=35, adjust=False).mean()
    company.dropna(inplace=False)    
    df = pd.concat([df, company],ignore_index=True)

##### Detectar Pivots/Fractals y Canales Bajistas y Alcistas, a travez de cruce de medias

In [5]:
ord=20
ord2=10
ord3=7
#for ticker in tickers:
#print("====", ticker)
#dfpl = df[(df.companyName==ticker)] #df.query("companyName=='AAPL'")
#max_idx = argrelextrema(df['Close'].values, np.greater, order=ord)[0]
#min_idx = argrelextrema(df['Close'].values, np.less, order=ord)[0]

#max_idx2 = argrelextrema(df['Close'].values, np.greater, order=ord2)[0]
#min_idx2 = argrelextrema(df['Close'].values, np.less, order=ord2)[0]


max_idx = argrelextrema(df['High'].values, np.greater, order=ord)[0]
min_idx = argrelextrema(df['Low'].values, np.less, order=ord)[0]

max_idx2 = argrelextrema(df['High'].values, np.greater, order=ord2)[0]
min_idx2 = argrelextrema(df['Low'].values, np.less, order=ord2)[0]

max_idx3 = argrelextrema(df['High'].values, np.greater, order=ord3)[0]
min_idx3 = argrelextrema(df['Low'].values, np.less, order=ord3)[0]

#print(max_idx)
#print(min_idx)
#df['pivotHigh'] = np.nan
#df['pivotLow'] = np.nan
#df['isPivot'] = np.nan
#df['isPivot'] = np.nan
# Aplicar el cálculo solo a los índices en la lista
df.loc[max_idx, 'pivotHigh'] = df['High']+1e-3
df.loc[min_idx, 'pivotLow'] = df['Low']-(1e-3)
df.loc[max_idx, 'isPivot'] = 1
df.loc[min_idx, 'isPivot'] = 2

df.loc[max_idx2, 'pivotHigh2'] = df['High']+1e-3
df.loc[min_idx2, 'pivotLow2'] = df['Low']-(1e-3)
df.loc[max_idx2, 'isPivot2'] = 1
df.loc[min_idx2, 'isPivot2'] = 2

df.loc[max_idx3, 'pivotHigh3'] = df['High']+1e-3
df.loc[min_idx3, 'pivotLow3'] = df['Low']-(1e-3)
df.loc[max_idx3, 'isPivot3'] = 1
df.loc[min_idx3, 'isPivot3'] = 2

In [6]:
# Señal: 1 = compra, -1 = venta, 0 = nada
#df['prev_SMA35'] = df['SMA35'].shift(1)
#df['prev_SMA50'] = df['SMA50'].shift(1)
df['prev_EMA35'] = df['EMA35'].shift(1)
df['prev_EMA50'] = df['EMA50'].shift(1)

#Cruce de medias
#df['cruce_medias'] = 0
#df.loc[(df['prev_SMA35'] < df['prev_SMA50']) & (df['SMA35'] > df['SMA50']), 'cruce_medias'] = 1  # Golden Cross (Compra)
#df.loc[(df['prev_SMA35'] > df['prev_SMA50']) & (df['SMA35'] < df['SMA50']), 'cruce_medias'] = -1 # Death Cross (Venta)

df['cruce_medias'] = 0
df.loc[(df['prev_EMA35'] < df['prev_EMA50']) & (df['EMA35'] > df['EMA50']), 'cruce_medias'] = 1  # Golden Cross (Compra)
df.loc[(df['prev_EMA35'] > df['prev_EMA50']) & (df['EMA35'] < df['EMA50']), 'cruce_medias'] = -1 # Death Cross (Venta)

#drop columnas
df.drop({'prev_EMA35','prev_EMA50'}, axis=1, inplace=True)


In [7]:
#Exportar tiempo Tendencia Alcista Bajista
path = 'data/tba.txt'
folder = os.path.dirname(path)

# Crear carpeta si no existe
if not os.path.exists(folder):
    os.makedirs(folder)

# Eliminar el archivo si ya existe
if os.path.exists(path):
    os.remove(path)
else:
    print("File does not exist. File needs to be created.")

# Exportar DataFrame a archivo de texto
df.to_csv(path, header=True, index=None, sep='\t', mode='w')


##### Funciones

In [8]:
#Funcion obtener vela entrada
def obtEntrada(dfpl,i,j, idvelafintend):
    indiceFinal=0
    indiceFinal2=0
    if (dfpl["cruce_medias"][i]==1): #ALCISTA               
        #ALZA, velas por encima de promedios moviles
        #ultimo high por encima y ultimo low cerca a los promedios
        #Obtener Siguiente Low
        siguiente_L = dfpl[(dfpl.index>=j) & (dfpl.index<idvelafintend) & ((dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2) | (dfpl["isPivot3"]==2))].head(1)        
        if (siguiente_L.shape[0]>0):         
            #if (((siguiente_L.iloc[0]['Low']-siguiente_L.iloc[0]['EMA35'])<1.5) | ((siguiente_L.iloc[0]['adx'])>20)):
            if (
                    ((siguiente_L.iloc[0]['isPivot']==2) | (siguiente_L.iloc[0]['isPivot2']==2))
                &
                    ((siguiente_L.iloc[0]['Low']-siguiente_L.iloc[0]['EMA35'])<=siguiente_L.iloc[0]['ATR']*2)
                &
                    ((siguiente_L.iloc[0]['Low']-siguiente_L.iloc[0]['EMA35'])>=-(siguiente_L.iloc[0]['ATR']*2))
                &
                    (siguiente_L.iloc[0]['adx']>10)
            ):
                indiceFinal = siguiente_L.index[0]
                indiceFinal2 = siguiente_L.index[0]
            elif (                
                    (siguiente_L.iloc[0]['isPivot3']==2)
                &
                    ((siguiente_L.iloc[0]['Low']-siguiente_L.iloc[0]['EMA35'])<=siguiente_L.iloc[0]['ATR']*2)
                &
                    ((siguiente_L.iloc[0]['Low']-siguiente_L.iloc[0]['EMA35'])>=-(siguiente_L.iloc[0]['ATR']*2))
                &
                    (siguiente_L.iloc[0]['adx']>25)
            ):
                indiceFinal = siguiente_L.index[0]
                indiceFinal2 = siguiente_L.index[0]
            #CASOS donde hubo salto de ATR
            elif (
                    ((siguiente_L.iloc[0]['isPivot']==2) | (siguiente_L.iloc[0]['isPivot2']==2))
                &
                    ((siguiente_L.iloc[0]['Low']<siguiente_L.iloc[0]['EMA35']))
                &
                    ((siguiente_L.iloc[0]['Low']<siguiente_L.iloc[0]['EMA35']))
                &
                    (siguiente_L.iloc[0]['adx']>15)
                &
                    (siguiente_L.iloc[0]['ATR']>siguiente_L.iloc[0]['EMA35_ATR']*1.15)
            ):
                indiceFinal = siguiente_L.index[0]
                indiceFinal2 = siguiente_L.index[0]

            else:
                #si es cero ir a la siguiente entrada
                indiceFinal2prev = siguiente_L.index[0]
                siguiente_L2 = dfpl[(dfpl.index>=indiceFinal2prev) & (dfpl.index<idvelafintend) & ((dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2) | (dfpl["isPivot3"]==2) )].head(1)
                if (siguiente_L2.shape[0]>0):
                    indiceFinal2 = siguiente_L2.index[0]
                else:
                    indiceFinal2 = siguiente_L.index[0]
                
    elif (dfpl["cruce_medias"][i]==-1): #BAJISTA                    
        #BAJA, velas por debajo de promedios moviles
        #ultimo low por debajo y ultimo high cerca a los promedios
        #Obtener Siguiente High
        siguiente_H = dfpl[(dfpl.index>=j) & (dfpl.index<idvelafintend) & ((dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1) | (dfpl["isPivot3"]==1))].head(1)        
        if (siguiente_H.shape[0]>0):         
            #if (((siguiente_H.iloc[0]['EMA35']-siguiente_H.iloc[0]['High'])<1.5) | ((siguiente_H.iloc[0]['adx'])>20)):
            if (
                    ((siguiente_H.iloc[0]['isPivot']==1) | (siguiente_H.iloc[0]['isPivot2']==1))
                    &
                    ((siguiente_H.iloc[0]['EMA35']-siguiente_H.iloc[0]['High'])<=siguiente_H.iloc[0]['ATR']*2) 
                    &
                    ((siguiente_H.iloc[0]['EMA35']-siguiente_H.iloc[0]['High'])>=-(siguiente_H.iloc[0]['ATR']*2))
                    &
                    (siguiente_H.iloc[0]['adx']>10)
                ):
                indiceFinal = siguiente_H.index[0]
                indiceFinal2 = siguiente_H.index[0]
            elif (
                    (siguiente_H.iloc[0]['isPivot3']==1)
                    &
                    ((siguiente_H.iloc[0]['EMA35']-siguiente_H.iloc[0]['High'])<=siguiente_H.iloc[0]['ATR']*2) 
                    &
                    ((siguiente_H.iloc[0]['EMA35']-siguiente_H.iloc[0]['High'])>=-(siguiente_H.iloc[0]['ATR']*2))
                    &
                    (siguiente_H.iloc[0]['adx']>25)
                ):
                indiceFinal = siguiente_H.index[0]
                indiceFinal2 = siguiente_H.index[0]
            #CASOS donde hubo salto de ATR
            elif (
                    ((siguiente_H.iloc[0]['isPivot']==1) | (siguiente_H.iloc[0]['isPivot2']==1))
                    &
                    ((siguiente_H.iloc[0]['EMA35']<siguiente_H.iloc[0]['High'])) 
                    &
                    ((siguiente_H.iloc[0]['EMA35']<siguiente_H.iloc[0]['High']))
                    &
                    (siguiente_H.iloc[0]['adx']>15)
                    &
                    (siguiente_H.iloc[0]['ATR']>siguiente_H.iloc[0]['EMA35_ATR']*1.15)
                ):
                indiceFinal = siguiente_H.index[0]
                indiceFinal2 = siguiente_H.index[0]
            else:
                #si es cero ir a la siguiente entrada
                indiceFinal2prev = siguiente_H.index[0]
                siguiente_H2 = dfpl[(dfpl.index>=indiceFinal2prev) & (dfpl.index<idvelafintend) & ((dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1) | (dfpl["isPivot3"]==1))].head(1)
                if siguiente_H2.shape[0]>0:
                    indiceFinal2 = siguiente_H2.index[0]
                else:
                    indiceFinal2 = siguiente_H.index[0]
    return indiceFinal, indiceFinal2

In [9]:
#Funcion obtener vela entrada
def obtEntrada2(dfpl,i,j, idvelafintend):
    indiceFinal=0
    indiceFinal2=0
    if (dfpl["cruce_medias"][i]==1): #ALCISTA               
        #ALZA, velas por encima de promedios moviles
        #ultimo high por encima y ultimo low cerca a los promedios
        #Obtener Siguiente Low
        siguiente_L = dfpl[(dfpl.index>=j) & (dfpl.index<=idvelafintend) & ((dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2))].head(1)        
        if (siguiente_L.shape[0]>0):         
            #if (((siguiente_L.iloc[0]['Low']-siguiente_L.iloc[0]['EMA35'])<1.5) | ((siguiente_L.iloc[0]['adx'])>20)):
            if (                
                    ((siguiente_L.iloc[0]['Low']-siguiente_L.iloc[0]['EMA35'])<=siguiente_L.iloc[0]['ATR']*2)
                &
                    ((siguiente_L.iloc[0]['Low']-siguiente_L.iloc[0]['EMA35'])>=-(siguiente_L.iloc[0]['ATR']*2))
                &
                    (siguiente_L.iloc[0]['adx']>10)
            ):
                indiceFinal = siguiente_L.index[0]
                indiceFinal2 = siguiente_L.index[0]
            else:
                #si es cero ir a la siguiente entrada
                indiceFinal2prev = siguiente_L.index[0]
                siguiente_L2 = dfpl[(dfpl.index>indiceFinal2prev) & (dfpl.index<=idvelafintend) & ((dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2))].head(1)
                if (siguiente_L2.shape[0]>0):
                    indiceFinal2 = siguiente_L2.index[0]
                else:
                    indiceFinal2 = siguiente_L.index[0]
                
    elif (dfpl["cruce_medias"][i]==-1): #BAJISTA                    
        #BAJA, velas por debajo de promedios moviles
        #ultimo low por debajo y ultimo high cerca a los promedios
        #Obtener Siguiente High
        siguiente_H = dfpl[(dfpl.index>=j) & (dfpl.index<=idvelafintend) & ((dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1))].head(1)        
        if (siguiente_H.shape[0]>0):         
            #if (((siguiente_H.iloc[0]['EMA35']-siguiente_H.iloc[0]['High'])<1.5) | ((siguiente_H.iloc[0]['adx'])>20)):
            if (
                    
                    ((siguiente_H.iloc[0]['EMA35']-siguiente_H.iloc[0]['High'])<=siguiente_H.iloc[0]['ATR']*2) 
                    &
                    ((siguiente_H.iloc[0]['EMA35']-siguiente_H.iloc[0]['High'])>=-(siguiente_H.iloc[0]['ATR']*2))
                    &
                    (siguiente_H.iloc[0]['adx']>10)
                ):
                indiceFinal = siguiente_H.index[0]
                indiceFinal2 = siguiente_H.index[0]
            else:
                #si es cero ir a la siguiente entrada
                indiceFinal2prev = siguiente_H.index[0]
                siguiente_H2 = dfpl[(dfpl.index>indiceFinal2prev) & (dfpl.index<=idvelafintend) & ((dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1))].head(1)
                if siguiente_H2.shape[0]>0:
                    indiceFinal2 = siguiente_H2.index[0]
                else:
                    indiceFinal2 = siguiente_H.index[0]
    return indiceFinal, indiceFinal2

In [10]:
#Funcion obtener vela entrada
def obtEntrada_old(dfpl,i,j, idvelafintend):
    indiceFinal=0
    indiceFinal2=0
    if (dfpl["cruce_medias"][i]==1): #ALCISTA               
        #ALZA, velas por encima de promedios moviles
        #ultimo high por encima y ultimo low cerca a los promedios
        #Obtener Siguiente Low
        siguiente_L = dfpl[(dfpl.index>=j) & (dfpl.index<=idvelafintend) & ((dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2))].head(1)        
        if (siguiente_L.shape[0]>0):         
            #if (((siguiente_L.iloc[0]['Low']-siguiente_L.iloc[0]['EMA35'])<1.5) | ((siguiente_L.iloc[0]['adx'])>20)):
            if (                
                    ((siguiente_L.iloc[0]['Low']-siguiente_L.iloc[0]['EMA35'])<=siguiente_L.iloc[0]['ATR']*2)
                &
                    ((siguiente_L.iloc[0]['Low']-siguiente_L.iloc[0]['EMA35'])>=-(siguiente_L.iloc[0]['ATR']*2))
                &
                    (siguiente_L.iloc[0]['adx']>10)
            ):
                indiceFinal = siguiente_L.index[0]
                indiceFinal2 = siguiente_L.index[0]
            else:
                #si es cero ir a la siguiente entrada
                indiceFinal2prev = siguiente_L.index[0]
                siguiente_L2 = dfpl[(dfpl.index>indiceFinal2prev) & (dfpl.index<=idvelafintend) & ((dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2) )].head(1)
                if (siguiente_L2.shape[0]>0):
                    indiceFinal2 = siguiente_L2.index[0]
                else:
                    indiceFinal2 = siguiente_L.index[0]
                
    elif (dfpl["cruce_medias"][i]==-1): #BAJISTA                    
        #BAJA, velas por debajo de promedios moviles
        #ultimo low por debajo y ultimo high cerca a los promedios
        #Obtener Siguiente High
        siguiente_H = dfpl[(dfpl.index>=j) & (dfpl.index<=idvelafintend) & ((dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1))].head(1)        
        if (siguiente_H.shape[0]>0):         
            #if (((siguiente_H.iloc[0]['EMA35']-siguiente_H.iloc[0]['High'])<1.5) | ((siguiente_H.iloc[0]['adx'])>20)):
            if (
                    ((siguiente_H.iloc[0]['EMA35']-siguiente_H.iloc[0]['High'])<=siguiente_H.iloc[0]['ATR']*2) 
                    &
                    ((siguiente_H.iloc[0]['EMA35']-siguiente_H.iloc[0]['High'])>=-(siguiente_H.iloc[0]['ATR']*2))
                    &
                    (siguiente_H.iloc[0]['adx']>10)
                ):
                indiceFinal = siguiente_H.index[0]
                indiceFinal2 = siguiente_H.index[0]
            else:
                #si es cero ir a la siguiente entrada
                indiceFinal2prev = siguiente_H.index[0]
                siguiente_H2 = dfpl[(dfpl.index>indiceFinal2prev) & (dfpl.index<=idvelafintend) & ((dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1))].head(1)
                if siguiente_H2.shape[0]>0:
                    indiceFinal2 = siguiente_H2.index[0]
                else:
                    indiceFinal2 = siguiente_H.index[0]
    return indiceFinal, indiceFinal2

In [11]:
#Funcion revisar Velas
def revisarVelas(dfpl, indiceFinal, i, tipo_pivot):
    ind_trendHL=0
    if tipo_pivot==1:
        ultimos_2H = dfpl[(dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1)].loc[:indiceFinal].tail(2)
        ultimos_2L = dfpl[(dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2)].loc[:indiceFinal].tail(2)
    else:
        ultimos_2H = dfpl[(dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1) | (dfpl["isPivot3"]==1)].loc[:indiceFinal].tail(2)
        ultimos_2L = dfpl[(dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2) | (dfpl["isPivot3"]==2)].loc[:indiceFinal].tail(2)

    #print("inicio")
    #print(ultimos_2H.index[0])
    #print("fin")
        
    if (ultimos_2H.shape[0]==2 & ultimos_2L.shape[0]==2):
        #print("h1")
        penultimo_valorH = ultimos_2H.iloc[0]['High']
        ultimo_valorH = ultimos_2H.iloc[1]['High']
        penultimo_valorL = ultimos_2L.iloc[0]['Low']
        ultimo_valorL = ultimos_2L.iloc[1]['Low']

        #print("idpenultimoH:",  ultimos_2H.index[0], ", penultimo_valorH:", penultimo_valorH, "idultimoH:",  ultimos_2H.index[1], ", ultimo_valorH:", ultimo_valorH)
        #print("idpenultimoL:",  ultimos_2L.index[0], ", penultimo_valorL:", penultimo_valorL, "idultimoH:",  ultimos_2L.index[1], ", ultimo_valorL:", ultimo_valorL)

        #En caso de tendencia ALCISTA
        #tomar los 2 ultimos HH – Higher High (Maximos mas Altos)
        #tomar los 2 ultimos HL – Higher Low (Maximos mas bajos)
        if (dfpl["cruce_medias"][i]==1): #ALCISTA
            if ((tipo_pivot==1) & ((ultimo_valorH>penultimo_valorH) | (ultimo_valorL>penultimo_valorL))):
                ind_trendHL=1
            elif ((tipo_pivot==2) & (ultimo_valorH>penultimo_valorH) & (ultimo_valorL>penultimo_valorL)):
                ind_trendHL=1

        #En caso de tendencia BAJISTA
        #tomar los 2 ultimos LH – Lower High (Minimos mas Altos)
        #tomar los 2 ultimos LL – Lower Low (Minimos mas bajos)
        elif (dfpl["cruce_medias"][i]==-1): #BAJISTA
            if ((tipo_pivot==1) & ((ultimo_valorH<penultimo_valorH) | (ultimo_valorL<penultimo_valorL))):
                ind_trendHL=1
            elif ((tipo_pivot==2) & (ultimo_valorH<penultimo_valorH) & (ultimo_valorL<penultimo_valorL)):
                ind_trendHL=1
    return ind_trendHL

In [12]:
#Funcion revisar Velas
def revisarVelas_old(dfpl, indiceFinal, i):
    ind_trendHL=0
    ultimos_2H = dfpl[(dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1)].loc[:indiceFinal].tail(1)
    ultimos_2L = dfpl[(dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2)].loc[:indiceFinal].tail(1)

    #print("inicio")
    #print(ultimos_2H.index[0])
    #print("fin")
        
    if (ultimos_2H.shape[0]==1 & ultimos_2L.shape[0]==1):
        #print("h1")
        penultimo_valorH = ultimos_2H.iloc[0]['High']
        ultimo_valorH = dfpl.iloc[indiceFinal]['High']
        penultimo_valorL = ultimos_2L.iloc[0]['Low']
        ultimo_valorL = dfpl.iloc[indiceFinal]['Low']

        #print("idpenultimoH:",  ultimos_2H.index[0], ", penultimo_valorH:", penultimo_valorH, "idultimoH:",  ultimos_2H.index[1], ", ultimo_valorH:", ultimo_valorH)
        #print("idpenultimoL:",  ultimos_2L.index[0], ", penultimo_valorL:", penultimo_valorL, "idultimoH:",  ultimos_2L.index[1], ", ultimo_valorL:", ultimo_valorL)

        #En caso de tendencia ALCISTA
        #tomar los 2 ultimos HH – Higher High (Maximos mas Altos)
        #tomar los 2 ultimos HL – Higher Low (Maximos mas bajos)
        if (dfpl["cruce_medias"][i]==1): #ALCISTA
            #print("h2")
            if ((ultimo_valorH>penultimo_valorH) | (ultimo_valorL>penultimo_valorL)):
                #print("h21")
                ind_trendHL=1

        #En caso de tendencia BAJISTA
        #tomar los 2 ultimos LH – Lower High (Minimos mas Altos)
        #tomar los 2 ultimos LL – Lower Low (Minimos mas bajos)
        elif (dfpl["cruce_medias"][i]==-1): #BAJISTA
            #print("h3")
            if ((ultimo_valorH<penultimo_valorH) | (ultimo_valorL<penultimo_valorL)):
                #print("h31")
                ind_trendHL=1
    return ind_trendHL

In [13]:
#Funcion revisar SLOPE
def obtSlope(dfpl,i,j,indiceFinal):
    #print("j:",j)
    ind_sl=sl35=sl50=sl=slH=slL=0
   
    #Inicio determinar slope que indica tendencia al ALZA o a la BAJA
    #El periodo de evaluacion estara dado por los 2 ultimos pivot
    
    dfHigher = dfpl[(dfpl.index<=indiceFinal) & (dfpl["isPivot"]==1)].tail(2)
    dfLower = dfpl[(dfpl.index<=indiceFinal) & (dfpl["isPivot"]==2)].tail(2)

    if ((len(dfHigher)>=2) and (len(dfLower)>=2)):
        idultHigher = dfHigher.index[-1]
        idpultHigher = dfHigher.index[-2]

        idultLower = dfLower.index[-1]
        idpultLower = dfLower.index[-2]

        # medias35 = dfpl.loc[j:indiceFinal].EMA35.values
        # idxmedias35 = dfpl.loc[j:indiceFinal].EMA35.index
        # medias50 = dfpl.loc[j:indiceFinal].EMA50.values
        # idxmedias50 = dfpl.loc[j:indiceFinal].EMA50.index
        # mediasClose = dfpl.loc[j:indiceFinal].Close.values
        # idxmediasClose = dfpl.loc[j:indiceFinal].Close.index

        mediasH = dfpl.loc[idpultHigher:idultHigher].High.values
        idxmediasH = dfpl.loc[idpultHigher:idultHigher].High.index

        mediasL = dfpl.loc[idpultLower:idultLower].Low.values
        idxmediasL = dfpl.loc[idpultLower:idultLower].Low.index

    
        slH, intercH, r_valueH, _, _ = stats.linregress(idxmediasH,mediasH)
        slL, intercL, r_valueL, _, _ = stats.linregress(idxmediasL,mediasL)
    
    # if ((len(medias35)>=2) and  (len(medias50)>=2)):
    #     sl35, interc35, r_value35, _, _ = stats.linregress(idxmedias35,medias35)
    #     sl50, interc50, r_value50, _, _ = stats.linregress(idxmedias50,medias50)
    #     sl, interc, r_value, _, _ = stats.linregress(idxmediasClose,mediasClose)
    #Fin slope

    #print ("sl35:", sl35,"sl50:", sl50,"sl:", sl)
    #print ("cruce_medias:", dfpl["cruce_medias"][i])
    
    if (dfpl["cruce_medias"][i]==1): #ALCISTA
        #Revision de pendiente
        if (slH>0 or slL>0 #Pendiente positiva
           #and np.isclose(sl35, sl50, atol=1e-5) #Verificar si son paralelas
           #and interc35>interc50 #La línea 35 está por encima de la línea 50
           ):
            ind_sl=1
    elif (dfpl["cruce_medias"][i]==-1): #BAJISTA
        #Revision de pendiente
        if (slH<0 or slL<0 #Pendiente negativa
            #and np.isclose(sl35, sl50, atol=1e-5) #Verificar si son paralelas
            #and interc50>interc35 #La línea 50 está por encima de la línea 35
           ):
            ind_sl=1
    return ind_sl, sl35, sl50, sl

In [14]:
def getBreakOutFinal(dfpl, i, dfprincipal):
    idbreakOutFinal = np.nan

    #Final de caso es siguiente Pivot o trailing stop loss con ATR
    if (dfpl["cruce_medias"][i]==1): #ALCISTA

        k=0
        while (k<=5):
            cnt2 = dfpl.query("index>@idBreakOutIni and isPivot==1").shape[0]
            if cnt2>0:        
                idbreakOutFinal = dfpl.query("index>@idBreakOutIni and isPivot==1").index[0]
                dfpl.loc[idbreakOutFinal,'isBreakOutFinal'] = 1
                k=6
            else:
                idfinal = dfpl.index[-1] 
                idfinal2 = idfinal+25
                if idfinal2 in dfprincipal[dfprincipal['companyName']==ticker].index:
                    dfpl2 = (dfprincipal[(dfprincipal.companyName==ticker)].loc[idfinal+1:idfinal2]).copy()
                    dfpl = pd.concat([dfpl, dfpl2],ignore_index=False)                           
                #else:                  
                #    dfpl['isBreakOutFinal'] = np.nan
                k=k+1
    
    elif (dfpl["cruce_medias"][i]==-1): #BAJISTA
        k=0
        while (k<=5):
            cnt2 = dfpl.query("index>@idBreakOutIni and isPivot==2").shape[0]
            if cnt2>0:        
                idbreakOutFinal = dfpl.query("index>@isBreakOutFinal and isPivot==2").index[0]
                dfpl.loc[idbreakOutFinal,'isBreakOutFinal'] = -1
                k=6
            else:
                idfinal = dfpl.index[-1] 
                idfinal2 = idfinal+25
                if idfinal2 in dfprincipal[dfprincipal['companyName']==ticker].index:
                    dfpl2 = (dfprincipal[(dfprincipal.companyName==ticker)].loc[idfinal+1:idfinal2]).copy()
                    dfpl = pd.concat([dfpl, dfpl2],ignore_index=False)                           
                #else:                  
                #    dfpl['isBreakOutFinal'] = np.nan
                k=k+1
    
    return idbreakOutFinal

##### Carga de Strike

In [17]:
#Leer el archivo de PREDICCION DE STRIKE
#Archivo anterior
ruta_archivo='data/prediccion_strike.txt'
if os.path.exists(ruta_archivo):
    # Cargar el archivo
    df_strike_pred_old = pd.read_csv(ruta_archivo, sep='\t')
    print("Archivo cargado correctamente.")
else:
    # Crear un DataFrame vacío
    df_strike_pred_old = pd.DataFrame()
    print("Archivo no existe. Se creó un DataFrame vacío.")

Archivo cargado correctamente.


In [ ]:
# Tabla que se mostrara en la APP como sugerencia de strike para opciones financieras
df_strike_pred_old

,Ticker,semana,Tag,strike_duration_mean,strike_price_mean,strike_price_q3
0,AAPL,>s4,long,33.000000,19.440000,17.718
1,AAPL,s1,long,3.866667,8.050000,4.016
2,AAPL,s1,short,3.190476,8.044286,2.640
3,AAPL,s2,long,12.333333,9.120000,6.200
4,AAPL,s2,short,9.857143,13.012857,8.384
5,AAPL,s3,long,17.666667,15.975000,9.960
6,AAPL,s4,long,22.000000,23.160000,23.160
7,SPY,s1,long,3.190476,8.484286,4.460
8,SPY,s1,short,3.666667,18.440000,6.230
9,SPY,s2,long,10.909091,17.873636,12.330


##### Obtencion de Casos

In [19]:
df_casos = pd.DataFrame()
df_casos_pivote = pd.DataFrame()
idcaso = 0
idcasopadre = 0
idcasohijo = 0
ticker2 = ""
#window=5
#backeval=75
posteval=0
backevalTrend=0
trendH = []
trendL = []
ispivot=0
dfprincipal = df.copy()
listruptura = [1,-1]

#Generar Trailing Stop con ATR
atr_mult_sl_1 = 1.2
atr_mult_sl_2 = 2
atr_mult_tp = 5
#dfprincipal['TS']=dfprincipal['Close'] - atr_mult_sl_1 * dfprincipal['ATR']

#Generar Trailing Stop con EMA20
#Opciones ITM/ATM a corto plazo (trading activo)	1.5% – 3% (0.015 – 0.03)
#Opciones más lejanas o swing (mayor horizonte)	2.5% – 5% (0.025 – 0.05)
#Basado en subyacente más estable (como índices)	1% – 2% (0.01 – 0.02
buffer_pct = 0.002  # 0.05%
filas = []
filas_pivote = []

for i, row in dfprincipal.iterrows():
    #if (i==6282):
    #Obtener la tendencia hasta donde se evaluaran casos
    idvelafintend=0
    tipo=""
    ticker=dfprincipal['companyName'][i]
    
    #if ticker=="AAPL":
    #Reinicio de casos por company
    if ticker2 != ticker:
        ticker2 = ticker
        idcaso = 0
        idcasopadre = 0
        
    if ((dfprincipal["cruce_medias"][i]==1) & (dfprincipal["companyName"][i]==ticker)): #ALCISTA
        velafintend = dfprincipal[(dfprincipal["cruce_medias"]==-1) & (dfprincipal.index>i) & (dfprincipal["companyName"][i]==ticker)].head(1)
        tipo="ALZA"
        position = 'long'
        if (velafintend.shape[0]>0):
            idvelafintend = velafintend.index[0]
    elif ((dfprincipal["cruce_medias"][i]==-1) & (dfprincipal["companyName"][i]==ticker)): #BAJISTA
        velafintend = dfprincipal[(dfprincipal["cruce_medias"]==1) & (dfprincipal.index>i) & (dfprincipal["companyName"][i]==ticker)].head(1)
        tipo="BAJA" 
        position = 'short'
        if (velafintend.shape[0]>0):
            idvelafintend = velafintend.index[0]
    
    if (tipo!=""):
        # if ((idvelafintend-i)<=75):
        #     posteval=i+75
        # elif ((idvelafintend-i)>75):
        #     #posteval=i+idvelafintend+10
        #     posteval=idvelafintend+10

        #Revisar prediccion STRIKE
        strike_calculado = None        
        strike_pred = 5 #valor predeterminado si no se ha calculado
        filtro = df_strike_pred_old.query("Ticker==@ticker and semana=='s1' and Tag==@position")
        if filtro.shape[0]>0:
            strike_calculado = filtro.iloc[0]["strike_price_q3"]
        if strike_calculado is None:
            strike_calculado = strike_pred

        print("strike_calculado:", strike_calculado)

        ind_sl = 0    
        df2 = pd.DataFrame()
        if idvelafintend==0:
            idvelafintend = dfprincipal[(dfprincipal.index>i) & (dfprincipal["companyName"][i]==ticker)].index[-1]
        print("==>hito carlos, i:", i, ", idvelafintend:", idvelafintend )
        df2 = (dfprincipal.query("index>=@i and index<@idvelafintend")).copy() #no tomar en cuenta el ultimo registro porque ya es cambio de tendencia
        
        iniEval = i            
        idcasohijo = 0
        for j, row in df2.iterrows():                
            if (j==iniEval):
                print ("j:", j, ", iniEval:", iniEval)
                dfpl = pd.DataFrame()
                #dfpl = (df[(df.companyName==ticker)].loc[i-backeval:posteval]).copy()
                dfpl = (df[(df.companyName==ticker)]).copy()
                idBreakOutIni, idBreakOutIni2 = obtEntrada(dfpl, i, j, idvelafintend)
                if idBreakOutIni2==j: idBreakOutIni2+=1
                print("===obteniendo entrada===")
                print ("idBreakOutIni:", idBreakOutIni)
                print ("idBreakOutIni2:", idBreakOutIni2)
                print("===obteniendo entrada===")
                if idBreakOutIni>0:
                    #revisar si es pivot3
                    tipo_pivot=1
                    if ((dfpl["isPivot3"][idBreakOutIni]==1) | (dfpl["isPivot3"][idBreakOutIni]==2)) & pd.isna(dfpl["isPivot2"][idBreakOutIni]):
                        tipo_pivot=2
                    ind_trendHL = revisarVelas(dfpl, idBreakOutIni, i, tipo_pivot)   
                    #ind_sl, sl35, sl50, sl = obtSlope(dfpl,i,j,idBreakOutIni)
                    adx = dfpl.loc[idBreakOutIni, "adx"]
                    #if ( (((ind_trendHL>0) or (ind_sl>0)) and (dfpl.loc[idBreakOutIni, "adx"]>15)) or ((dfpl.loc[idBreakOutIni, "adx"]>25) and (dfpl.loc[idBreakOutIni, "ATR"]>dfpl.loc[idBreakOutIni, "EMA35_ATR"]))):
                    #if ( ((ind_trendHL>0) and (dfpl.loc[idBreakOutIni, "adx"]>15)) or ((dfpl.loc[idBreakOutIni, "adx"]>25) and (dfpl.loc[idBreakOutIni, "ATR"]>dfpl.loc[idBreakOutIni, "EMA35_ATR"]))):
                    if ( (ind_trendHL>0) or ((dfpl.loc[idBreakOutIni, "adx"]>15) and (dfpl.loc[idBreakOutIni, "ATR"]>dfpl.loc[idBreakOutIni, "EMA35_ATR"]*1.15))):
                        #INSERT CASO
                        idcaso = idcaso + 1
                        idcasohijo = idcasohijo + 1
                        if idcasohijo==1: idcasopadre = idcasopadre + 1
                        #Recorrer para obtener la salida
                        df3 = (dfprincipal.query("index>=@idBreakOutIni and companyName==@ticker")).copy()
                        #price = dfprincipal.loc[idBreakOutIni, 'EMA5'] #Revisar que EMA es la apropiada                            
                        price = dfprincipal.loc[idBreakOutIni, 'Close'] #carlos 1707
                        atr = dfprincipal.loc[idBreakOutIni, 'ATR']
                        atr2 = dfprincipal.loc[idBreakOutIni, 'ATR2']
                        datetime = dfprincipal.loc[idBreakOutIni, 'Datetime']

                        # Variables de control
                        #position = None  # 'long' o 'short'
                        
                        entry_price = trailing_stop = None
                        
                        if (tipo=="ALZA"):
                            position = 'long'
                            entry_price = price
                            entry_date = datetime                                
                            #trailing_stop = price * (1 - buffer_pct)
                            trailing_stop = price - atr_mult_sl_1 * atr
                            #take_profit_pred = (price+strike_calculado) + atr_mult_sl * atr
                            take_profit_pred = price+strike_calculado+atr_mult_sl_2*atr2
                            dfprincipal.loc[i,'ind_posicion']=1
                            dfprincipal.loc[idBreakOutIni,'isBreakOutIni']=1
                            print(f"ENTRADA LONG en {price:.2f} el {datetime} (TSL: {trailing_stop:.2f}) idBreakOutIni: {idBreakOutIni}")
                        
                        elif (tipo=="BAJA"):
                            position = 'short'
                            entry_price = price
                            entry_date = datetime                                
                            #trailing_stop = price * (1 + buffer_pct)
                            trailing_stop = price + atr_mult_sl_1 * atr
                            #take_profit_pred = (price-strike_calculado) - atr_mult_sl * atr
                            take_profit_pred = price-strike_calculado-atr_mult_sl_2*atr2
                            dfprincipal.loc[i,'ind_posicion']=-1
                            dfprincipal.loc[idBreakOutIni,'isBreakOutIni']=-1
                            print(f"ENTRADA SHORT en {price:.2f} el {datetime} (TSL: {trailing_stop:.2f}) idBreakOutIni: {idBreakOutIni}")

                        datetime2 = None
                        price2 = None
                        id2 = 0
                        #Revisar puntos mas altos, para obtener el STRIKE minimo
                        if position == 'long': #ALCISTA
                            cnt2 = dfpl.query("index>@idBreakOutIni and isPivot==1").shape[0]
                            if cnt2>0:
                                id2 = dfpl.query("index>@idBreakOutIni and isPivot==1").index[0]
                        elif position == 'short': #BAJISTA
                            cnt2 = dfpl.query("index>@idBreakOutIni and isPivot==2").shape[0]
                            if cnt2>0:
                                id2 = dfpl.query("index>@idBreakOutIni and isPivot==2").index[0]

                        exit_date_pivote = exit_price_pivote =None

                        if (id2>0):
                            exit_date_pivote = dfpl.loc[id2, 'Datetime']
                            exit_price_pivote = dfpl.loc[id2, 'Close']

                        fila_pivote = [ticker, idcaso,entry_price, exit_price_pivote, entry_date, exit_date_pivote, position, adx, atr, idcasopadre, idcasohijo, trailing_stop, strike_calculado] 
                        
                        #ticker, idcaso, price, "ExitPrice", datetime, "ExitTime", position
                        fila = [ticker, idcaso,entry_price, None, entry_date, None, position, adx, atr, idcasopadre, idcasohijo, trailing_stop, strike_calculado]
                        dfprincipal.loc[idBreakOutIni,'trailing_stop']=trailing_stop #carlos tmp
                        
                        #estrategia para hallar la salida: usar stop loss estatico, si precio supera el strike calculado usar trailing stop
                        #STOP LOSS ESTATICO
                        tipo_stop = 1

                        #Obtener Salida utilizando Trailing Stop ATR
                        for k, row3 in df3.iterrows():
                            #price = dfprincipal.loc[k, 'EMA5']
                            price = dfprincipal.loc[k, 'Close']
                            atr = dfprincipal.loc[k, 'ATR']
                            datetime = dfprincipal.loc[k, 'Datetime']

                            #revisar si tipo cambia a STOP LOSS DINAMICO
                            if position == 'long' and price >= take_profit_pred:
                                tipo_stop = 2
                            elif position == 'short' and price <= take_profit_pred:
                                tipo_stop = 2
                                        
                            # Cerrar posición si el precio toca el trailing stop                                                               
                            if position == 'long' and price <= trailing_stop:
                                iniEval = k
                                dfprincipal.loc[k,'isBreakOutFinal']=1
                                print(f"SALIDA LONG en {price:.2f} el {datetime} (TSL: {trailing_stop:.2f})  idBreakOutFinal: {iniEval}")
                                #fila[3]=price
                                fila[3] = trailing_stop #+ (1e-3)
                                fila[5]=datetime
                                position = None
                                entry_price = trailing_stop
                                break

                            if position == 'short' and price >= trailing_stop:
                                iniEval = k
                                dfprincipal.loc[k,'isBreakOutFinal']=-1
                                print(f"SALIDA SHORT en {price:.2f} el {datetime} (TSL: {trailing_stop:.2f})  idBreakOutFinal: {iniEval}")
                                #fila[3]=price
                                fila[3] = trailing_stop #- (1e-3)
                                fila[5]=datetime
                                position = None
                                entry_price = trailing_stop 
                                break
                                
                            if tipo_stop == 2:
                                #Si hay posición abierta y es TRAILING STOP, actualizar nuevo stop (se incrementa multiplicador)
                                if position=="long":                                  
                                    #new_stop = price * (1 - buffer_pct)
                                    new_stop = price - atr_mult_sl_2 * atr
                                    if new_stop > trailing_stop:
                                        trailing_stop = new_stop
                                    
                                elif position=="short":                                    
                                    #new_stop = price * (1 + buffer_pct)
                                    new_stop = price + atr_mult_sl_2 * atr
                                    if new_stop < trailing_stop:
                                        trailing_stop = new_stop

                            dfprincipal.loc[k, 'trailing_stop'] = trailing_stop #carlos tmp
                            #dfprincipal.loc[k, 'trailing_stop2'] = trailing_stop2 #carlos tmp
                        filas.append(fila)
                        filas_pivote.append(fila_pivote)
                    else:
                        iniEval = idBreakOutIni2
                else:
                    iniEval = idBreakOutIni2

strike_calculado: 6.230000000000018
==>hito carlos, i: 118 , idvelafintend: 247
j: 118 , iniEval: 118
===obteniendo entrada===
idBreakOutIni: 129
idBreakOutIni2: 129
===obteniendo entrada===
ENTRADA SHORT en 452.74 el 2023-08-04 12:00:00 (TSL: 454.63) idBreakOutIni: 129
SALIDA SHORT en 438.71 el 2023-08-21 09:30:00 (TSL: 437.86)  idBreakOutFinal: 203
j: 203 , iniEval: 203
===obteniendo entrada===
idBreakOutIni: 210
idBreakOutIni2: 210
===obteniendo entrada===
ENTRADA SHORT en 439.64 el 2023-08-22 09:30:00 (TSL: 441.70) idBreakOutIni: 210
SALIDA SHORT en 441.92 el 2023-08-23 10:00:00 (TSL: 441.70)  idBreakOutFinal: 218
j: 218 , iniEval: 218
===obteniendo entrada===
idBreakOutIni: 0
idBreakOutIni2: 225
===obteniendo entrada===
j: 225 , iniEval: 225
===obteniendo entrada===
idBreakOutIni: 0
idBreakOutIni2: 226
===obteniendo entrada===
j: 226 , iniEval: 226
===obteniendo entrada===
idBreakOutIni: 0
idBreakOutIni2: 0
===obteniendo entrada===
strike_calculado: 4.460000000000036
==>hito carlo

In [20]:
#Convertir a dataframe la lista
columnas = ['Ticker', 'caso', 'EntryPrice', 'ExitPrice', 'EntryTime', 'ExitTime', 'Tag', 'ADX', 'ATR', 'casopadre', 'casohijo','trailingstop','strike_price_q3_old']
df_casos = pd.DataFrame(filas, columns=columnas)
df_casos_pivote = pd.DataFrame(filas_pivote, columns=columnas)

In [78]:
#df_casos[df_casos['Ticker']=='SPY'].tail(50)

In [22]:
df_casos['duration'] = np.where(df_casos['ExitTime'].notna(), (df_casos['ExitTime'] - df_casos['EntryTime']).dt.days, None)
df_casos['price_distance'] =  np.where((df_casos['ExitPrice'].notna()) & (df_casos['Tag']=="long"), df_casos['ExitPrice'] - df_casos['EntryPrice'],
                                       np.where((df_casos['ExitPrice'].notna()) & (df_casos['Tag']=="short"), 
                                                df_casos['EntryPrice'] - df_casos['ExitPrice'], None))

df_casos_pivote['duration'] = np.where(df_casos_pivote['ExitTime'].notna(), (df_casos_pivote['ExitTime'] - df_casos_pivote['EntryTime']).dt.days, None)
df_casos_pivote['price_distance'] =  np.where((df_casos_pivote['ExitPrice'].notna()) & (df_casos_pivote['Tag']=="long"), df_casos_pivote['ExitPrice'] - df_casos_pivote['EntryPrice'],
                                       np.where((df_casos_pivote['ExitPrice'].notna()) & (df_casos_pivote['Tag']=="short"), 
                                                df_casos_pivote['EntryPrice'] - df_casos_pivote['ExitPrice'], None))

In [23]:
df_casos_pivote['semana'] = np.where(df_casos_pivote['duration']<=7, 's1',
                            np.where((df_casos_pivote['duration']>7) & (df_casos_pivote['duration']<=14), 's2',
                            np.where((df_casos_pivote['duration']>14) & (df_casos_pivote['duration']<=21), 's3', 
                            np.where((df_casos_pivote['duration']>21) & (df_casos_pivote['duration']<=28), 's4','>s4')         
                                     )))

In [24]:
#df_casos_pivote[(df_casos_pivote["Ticker"]=="AAPL") & (df_casos_pivote["Tag"]=="long")].sort_values(by="price_distance", ascending=True)
# Mostrar todas las filas
#pd.reset_option('display.max_rows')
#pd.set_option('display.max_rows', None)
#df_casos_pivote[(df_casos_pivote["Ticker"]=="AAPL")].sort_values(by="price_distance", ascending=True)

In [25]:
# Agrupar y calcular promedio y quartil
df_strike_pred = df_casos_pivote[df_casos_pivote["price_distance"]>0].groupby(['Ticker', 'semana', 'Tag']).agg({
    'duration': ['mean'],
    'price_distance': ['mean', lambda x: x.quantile(0.20)]
}).reset_index()

In [26]:
# Renombrar columnas para mayor claridad
df_strike_pred.columns = ['Ticker', 'semana', 'Tag', 'strike_duration_mean', 'strike_price_mean', 'strike_price_q3']

In [79]:
#df_strike_pred

In [28]:
path = 'data/prediccion_strike.txt'
folder = os.path.dirname(path)

# Crear carpeta si no existe
if not os.path.exists(folder):
    os.makedirs(folder)

# Eliminar el archivo si ya existe
if os.path.exists(path):
    os.remove(path)
else:
    print("File does not exist. File needs to be created.")

# Exportar DataFrame a archivo de texto
df_strike_pred.to_csv(path, header=True, index=None, sep='\t', mode='w')

In [29]:
df_casos_comparacion = pd.merge(df_casos, df_casos_pivote[['EntryTime','Ticker','Tag', 'duration', 'price_distance','semana']], on=['EntryTime','Ticker','Tag'], how='left')

In [30]:
df_casos_comparacion = df_casos_comparacion.rename(columns={'duration_x':'duration_ts', 
                                                            'price_distance_x': 'price_distance_ts', 
                                                            'duration_y': 'duration_pivote',
                                                            'price_distance_y': 'price_distance_pivote'})

In [31]:
df_casos_comparacion = pd.merge(df_casos_comparacion, df_strike_pred, on=['Ticker','semana','Tag'], how='left')

In [32]:
#df_strike_pred[['Ticker','Tag','strike_price_q3']][df_strike_pred['semana']=='s1']
df_casos_comparacion = pd.merge(df_casos_comparacion, df_strike_pred[['Ticker','Tag','strike_price_q3']][df_strike_pred['semana']=='s1'], on=['Ticker','Tag'], how='left')

In [33]:
df_casos_comparacion = df_casos_comparacion.rename(columns={'strike_price_q3_x':'strike_price_q3', 
                                                            'strike_price_q3_y':'strike_price_q3_s1'})
#

In [34]:
#df_casos_comparacion[(df_casos_comparacion['Ticker']=='SPY') & (df_casos_comparacion['Tag']=='long')]

In [35]:
#df_casos_comparacion['dif_price'] = df_casos_comparacion['price_distance_pivote'] - df_casos_comparacion['price_distance_ts']
#df_casos_comparacion = df_casos_comparacion.drop(['dif_price'], axis=1)

In [36]:
#df_casos_comparacion[['Ticker','duration_ts','price_distance_ts','duration_pivote','price_distance_pivote','semana','strike_price_mean','strike_price_q3']]
df_casos_comparacion['comparacion'] = np.where(((df_casos_comparacion['strike_price_q3'].notna())
                                            & (df_casos_comparacion['price_distance_ts'].notna())),
                                            (df_casos_comparacion['price_distance_ts']-df_casos_comparacion['strike_price_q3']),np.nan)

df_casos_comparacion['comparacion_2'] = np.where(((df_casos_comparacion['strike_price_q3_s1'].notna())
                                            & (df_casos_comparacion['price_distance_ts'].notna())),
                                            (df_casos_comparacion['price_distance_ts']-df_casos_comparacion['strike_price_q3_s1']),np.nan)


In [80]:
#df_casos_comparacion

In [38]:
df_casos_comparacion['ind_exito'] = np.where(df_casos_comparacion['comparacion']>0,1,0)
df_casos_comparacion['ind_exito_2'] = np.where(df_casos_comparacion['comparacion_2']>0,1,0)

In [81]:
#df_casos_comparacion[df_casos_comparacion['ind_exito']>0] 
#df_casos_comparacion[(df_casos_comparacion['Ticker']=='AAPL') & (df_casos_comparacion['semana']=='s2') & (df_casos_comparacion['Tag']=='short') 
#                     & (df_casos_comparacion['price_distance_ts']>df_casos_comparacion['price_distance_pivote'])]

#df_casos_comparacion[['Ticker', 'Tag','EntryPrice','ExitPrice','price_distance_ts','strike_price_q3','strike_price_q3_s1','price_distance_pivote','semana']][(df_casos_comparacion['ind_exito']==1)]

In [41]:
#df_ratioprofit = df_casos_comparacion.groupby(['Ticker','semana','Tag'])[['price_distance_ts','price_distance_pivote']].sum().reset_index()
df_profit = df_casos_comparacion[(df_casos_comparacion['duration_pivote'].notna()) & (df_casos_comparacion['duration_ts'].notna())].groupby(
    ['Ticker','semana','Tag','ind_exito']).agg({
    'price_distance_ts': ['count','sum'],
    'price_distance_pivote': 'sum',
    'strike_price_q3': 'max'
}).reset_index()


df_profit_2 = df_casos_comparacion[(df_casos_comparacion['duration_pivote'].notna()) & (df_casos_comparacion['duration_ts'].notna())].groupby(
    ['Ticker','Tag','ind_exito_2']).agg({
    'price_distance_ts': ['count','sum'],
    'price_distance_pivote': 'sum',
    'strike_price_q3_s1': 'max'
}).reset_index()

In [82]:
#df_profit_2

In [43]:
# Renombrar columnas para mayor claridad
df_profit.columns = ['Ticker', 'semana', 'Tag', 'ind_exito','cantidad', 'price_distance_ts', 'price_distance_pivote', 'strike_price_q3']
df_profit_2.columns = ['Ticker', 'Tag', 'ind_exito_2','cantidad', 'price_distance_ts', 'price_distance_pivote', 'strike_price_q3_s1']

In [44]:
#df_pivot = df_ratioprofit.pivot_table(index=['Ticker', 'semana', 'Tag'], columns='cantidad', values='ind_exito', aggfunc='max')
df_ratioprofit = df_profit.pivot_table(index=['Ticker', 'semana', 'Tag'], columns='ind_exito', values=['cantidad','price_distance_ts','price_distance_pivote'])
df_ratioprofit_2 = df_profit_2.pivot_table(index=['Ticker', 'Tag','strike_price_q3_s1'], columns='ind_exito_2', values=['cantidad','price_distance_ts','price_distance_pivote'])


In [83]:
#df_ratioprofit_2

In [46]:
df_ratioprofit.columns = ['cnt_perdida','cnt_exito', 'price_pivote_perdida', 'price_pivote_exito','price_ts_perdida','price_ts_exito']
df_ratioprofit_2.columns = ['cnt_perdida','cnt_exito', 'price_pivote_perdida', 'price_pivote_exito','price_ts_perdida','price_ts_exito']

In [49]:
df_ratioprofit['winrate'] = np.where(df_ratioprofit['cnt_exito'].isna(),0,
                                     df_ratioprofit['cnt_exito']/(df_ratioprofit['cnt_perdida']+df_ratioprofit['cnt_exito']))

df_ratioprofit['ratio_price'] = np.where(df_ratioprofit['price_ts_exito'].isna(),0,
                                     df_ratioprofit['price_ts_exito']/(df_ratioprofit['price_pivote_exito']+ df_ratioprofit['price_pivote_perdida']))

df_ratioprofit_2['winrate'] = np.where(df_ratioprofit_2['cnt_exito'].isna(),0,
                                     df_ratioprofit_2['cnt_exito']/(df_ratioprofit_2['cnt_perdida']+df_ratioprofit_2['cnt_exito']))

df_ratioprofit_2['ratio_price'] = np.where(df_ratioprofit_2['price_ts_exito'].isna(),0,
                                     df_ratioprofit_2['price_ts_exito']/(df_ratioprofit_2['price_pivote_exito']+ df_ratioprofit_2['price_pivote_perdida']))
                                     

In [87]:
#df_ratioprofit_2

##### Revisar Grafico

In [76]:
#Este codigo es para revisar los casos eliminar para el proceso automatico
bokeh.io.reset_output()
#bokeh.io.output_notebook()
#CODIGO DE REVISION CARLOS
#dfpl = dfprincipal.iloc[500:600]

tickerSel = 'AAPL'
annio = 2025

dfpl = dfprincipal[(dfprincipal["companyName"]==tickerSel) & (dfprincipal["Datetime"].dt.year==annio)].copy()

# dfpl = dfprincipal[(dfprincipal["companyName"]=="AAPL") & (dfprincipal["Datetime"].dt.year==2025)].merge(df_casos_pivote[['Ticker', 'EntryTime', 'Tag', 'caso']], 
#                 left_on=['companyName', 'Datetime'],
#                 right_on=['Ticker', 'EntryTime'], 
#                 how='left')

# INICIO agregar columnas de df_casos
dfpl = pd.merge(dfpl,df_casos[['Ticker','caso','EntryTime','strike_price_q3_old','price_distance']], left_on=['companyName','Datetime'], 
                right_on=['Ticker','EntryTime'], how='left').set_index(dfpl.index)
dfpl.drop(['Ticker','EntryTime'], axis=1, inplace=True)
dfpl = pd.merge(dfpl,df_casos[['Ticker','caso','ExitTime','strike_price_q3_old','price_distance']], left_on=['companyName','Datetime'], 
                right_on=['Ticker','ExitTime'], how='left').set_index(dfpl.index)
dfpl['caso'] = np.where(dfpl['caso_x'].notna(),dfpl['caso_x'], dfpl['caso_y'])
dfpl['strike_price_q3_old'] = np.where(dfpl['strike_price_q3_old_x'].notna(),dfpl['strike_price_q3_old_x'], dfpl['strike_price_q3_old_y'])
dfpl['price_distance'] = np.where(dfpl['price_distance_x'].notna(),dfpl['price_distance_x'], dfpl['price_distance_y'])
dfpl.drop(['Ticker','ExitTime','caso_x','caso_y','strike_price_q3_old_x','strike_price_q3_old_y','price_distance_x','price_distance_y'], axis=1, inplace=True)
dfpl['price_distance'] = dfpl['price_distance'].astype(float)
#Usa .loc cuando estés trabajando con condiciones sobre columnas que pueden tener nulos
mask = dfpl['strike_price_q3_old'].notna() & dfpl['price_distance'].notna()
dfpl.loc[mask, 'exito']=0
dfpl.loc[ mask & (dfpl['price_distance']>=dfpl['strike_price_q3_old']) , 'exito'] = 1
# FIN agregar columnas de df_casos


#dfpl = dfprincipal[(dfprincipal["companyName"]=="AAPL")]
dfpl["Datetime_str"] = dfpl["Datetime"].astype(str)

p = figure(width=2500, height=550,
        title=f"Tendencia Bajista - Alcista {tickerSel} - {str(annio)}",
        background_fill_color="#efefef",
        tooltips=[("Index", "@index"), ("caso", "@caso"),("Datetime", "@Datetime_str"), ("Open", "@Open"), ("High","@High"), ("Low","@Low"), ("Close","@Close"),
                    ("EMA35", "@EMA35"),
                    ("EMA50", "@EMA50"),
                    ("adx", "@adx"),
                    ("ATR", "@ATR"),
                    ("ATR2", "@ATR2"),
                    ("EMA35_ATR", "@EMA35_ATR"),
                    ("trailing_stop", "@trailing_stop"),
                    ("CASO", "@caso"),
                    ("Strike Realizado", "@price_distance"),
                    ("Strike Prediccion", "@strike_price_q3_old")
                    #("SMA35", "@SMA35"),
                    #("SMA50", "@EMA50")
                    #("cdlengulfing","@cdlengulfing"), 
                    #("cdlhammer","@cdlhammer"), 
                    #("cdlmorningstar","@cdlmorningstar"), 
                    #("cdlpiercing","@cdlpiercing"), 
                    #("cdlclosingmarubozu","@cdlclosingmarubozu"), 
                    #("cdlmarubozu","@cdlmarubozu"), 
                    #("cdl3whitesoldiers","@cdl3whitesoldiers"), 
                    #("cdlharami","@cdlharami"), 
                    #("cdlharamicross","@cdlharamicross"), 
                    #("cdlinvertdhammer","@cdlinvertdhammer"), 
                    #("cdlladderbottom","@cdlladderbottom")
                 ]
        #tooltips=[("Index", "@index"), ("Open", "@Open"), ("High","@High"), ("Low","@Low"), ("Close","@Close")]
        )
#p.xaxis.major_label_orientation = 0.8 # radians
#p.x_range.range_padding = 0.05
#p.xaxis.axis_line_join = "bevel" # radians
#p.xaxis.axis_line_width = 2
#p.xaxis.major_label_overrides = {
#    i: date.strftime('%b %d %T') for i, date in zip(dfpl.index, dfpl["datetime"])
#}

p.segment("index", "High", "index","Low",  color="black", line_width=1, source=dfpl)

inc = dfpl.query("Close>Open")
dec = dfpl.query("Open>Close")

p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="red",
    line_color="red",    
    source=dec   
)

p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="green",
    line_color="green", 
    source=inc   
)



# p.line(
#    x="index", 
#    y="EMA20", 
#    color="pink",
#    legend_label="EMA20",
#    source=dfpl)

# p.line(
#    x="index", 
#    y="EMA10", 
#    color="yellow",
#    legend_label="EMA10",
#    source=dfpl)

# p.line(
#    x="index", 
#    y="EMA5", 
#    color="red",
#    legend_label="EMA5",
#    source=dfpl)
#
#
#p.line(
#    x="index", 
#    y="EMA40", 
#    color="blue",
#    legend_label="EMA40",
#    source=dfpl)


p.line(
    x="index", 
    y="EMA35", 
    color="green",
    legend_label="EMA35",
    source=dfpl)


p.line(
    x="index", 
    y="EMA50", 
    color="red",
    legend_label="EMA50",
    source=dfpl)

# p.line(
#     x="index", 
#     y="TS", 
#     color="purple",
#     legend_label="TS",
#     source=dfpl)

p.line(
    x="index", 
    y="trailing_stop", 
    color="black",
    legend_label="trailing_stop",
    source=dfpl)

# p.line(
#     x="index", 
#     y="trailing_stop2", 
#     color="brown",
#     legend_label="trailing_stop2",
#     source=dfpl)

#p.line(
#    x="index", 
#    y="SMA100", 
#    color="green",
#    legend_label="SMA100",
#    source=dfpl)
#
#p.line(
#    x="index", 
#    y="SMA200", 
#    color="purple",
#    legend_label="SMA200",
#    source=dfpl)
p.scatter(x="index", y="Low", marker="triangle", size=20,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="inicio1", source=dfpl[(dfpl.isBreakOutIni==1)])

p.scatter(x="index", y="High", marker="inverted_triangle", size=20,
           line_color="navy", fill_color="red", alpha=0.5, legend_label="inicio2", source=dfpl[(dfpl.isBreakOutIni==-1)])

p.scatter(x="index", y="High", marker="inverted_triangle", size=20,
           line_color="navy", fill_color="black", alpha=0.5, legend_label="fin1", source=dfpl[(dfpl.isBreakOutFinal==1)])

p.scatter(x="index", y="Low", marker="triangle", size=20,
           line_color="navy", fill_color="brown", alpha=0.5, legend_label="fin2", source=dfpl[(dfpl.isBreakOutFinal==-1)])

p.scatter(x="index", y="pivotHigh", marker="circle", size=5,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="Cambio Tendencia Bajista1", source=dfpl[(dfpl.isPivot==1)])

p.scatter(x="index", y="pivotLow", marker="circle", size=5,
           line_color="navy", fill_color="green", alpha=0.5, legend_label="Cambio Tendencia Alcista1", source=dfpl[(dfpl.isPivot==2)])

p.scatter(x="index", y="pivotHigh2", marker="diamond", size=8,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="Cambio Tendencia Bajista2", source=dfpl[(dfpl.isPivot2==1) ])

p.scatter(x="index", y="pivotLow2", marker="diamond", size=8,
           line_color="navy", fill_color="green", alpha=0.5, legend_label="Cambio Tendencia Alcista2", source=dfpl[(dfpl.isPivot2==2) ])

p.scatter(x="index", y="pivotHigh3", marker="plus", size=8,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="Cambio Tendencia Bajista3", source=dfpl[(dfpl.isPivot3==1) & (dfpl.isPivot.isna()) & (dfpl.isPivot2.isna()) ])

p.scatter(x="index", y="pivotLow3", marker="plus", size=8,
           line_color="navy", fill_color="green", alpha=0.5, legend_label="Cambio Tendencia Alcista3", source=dfpl[(dfpl.isPivot3==2) & (dfpl.isPivot.isna()) & (dfpl.isPivot2.isna()) ])


p.yaxis[0].formatter = NumeralTickFormatter(format="$0.00")
p.xaxis.axis_label = "Fecha"
p.yaxis.axis_label = "Precio"
p.legend.location="top_left"
p.legend.click_policy="hide"

#Render linea vertical
alza = (dfpl[(dfpl.ind_posicion==1)].index).tolist()
baja = (dfpl[(dfpl.ind_posicion==-1)].index).tolist()
#vline=Span(source=inicio,dimension='height', line_color='blue',line_width=0.8, line_dash_offset= 0, line_dash='dashed', name="hola esto es una prueba", level='annotation', tags= ['square'])
#p.renderers.extend([vline])

p.vspan(
    x=alza,
    line_color="blue"
)

p.vspan(
    x=baja,
    line_color="red"
)

p.xaxis.visible = False

dfpl_cols = ColumnDataSource(dfpl[dfpl['exito'].notna()])


labels = LabelSet(x='index', y='High', text='exito', x_offset=5, y_offset=5, source=dfpl_cols)

p.add_layout(labels)

#ADX
adx = figure(x_axis_type="datetime", width=2500, height=150, tooltips = [("minus_di", "@minus_di"), ("plus_di", "@plus_di"), ("adx", "@adx"),("Datetime", "@Datetime_str")],
background_fill_color="#efefef",x_range=p.x_range)

#adx.x_range.range_padding = 0.05
adx.line(
    x="index", 
    y="minus_di", 
    color="red",
    legend_label="minus_di",
    source=dfpl)

adx.line(
    x="index", 
    y="plus_di", 
    color="green",
    legend_label="plus_di",
    source=dfpl)

adx.line(
    x="index", 
    y="adx", 
    color="black",
    legend_label="adx",
    source=dfpl)

#adx.xaxis.axis_label = "Fecha"
#adx.xaxis.major_label_overrides = {
#    i: date.strftime('%b %d %T') for i, date in zip(dfpl.index, dfpl["datetime"])
#}
adx.yaxis[0].formatter = NumeralTickFormatter(format="0,0")

adx.hspan(
    y=[20],
    line_width=[2], line_color="gray",
)

adx.xaxis.visible = False
#Fin ADX

#ATR2
atr2 = figure(x_axis_type="datetime", width=2500, height=150, tooltips = [("ATR2", "@ATR2"),("Datetime", "@Datetime_str")],
background_fill_color="#efefef",x_range=p.x_range)

atr2.xaxis.axis_label = "Fecha"
atr2.xaxis.major_label_orientation = 0.8 # radians
atr2.x_range.range_padding = 0.05
#p.xaxis.axis_line_join = "bevel" # radians
atr2.xaxis.axis_line_width = 2
atr2.xaxis.major_label_overrides = {
    i: date.strftime('%b %d %T') for i, date in zip(dfpl.index, dfpl["Datetime"])
}

atr2.line(
    x="index", 
    y="ATR2", 
    color="black",
    legend_label="ATR2",
    source=dfpl)


atr2.yaxis[0].formatter = NumeralTickFormatter(format="0,0")
atr2.xaxis.visible = False
#Fin ATR2

#ATR
atr = figure(x_axis_type="datetime", width=2500, height=200, tooltips = [("ATR", "@ATR"),("Datetime", "@Datetime_str")],
background_fill_color="#efefef",x_range=p.x_range)

atr.xaxis.axis_label = "Fecha"
atr.xaxis.major_label_orientation = 0.8 # radians
atr.x_range.range_padding = 0.05
#p.xaxis.axis_line_join = "bevel" # radians
atr.xaxis.axis_line_width = 2
atr.xaxis.major_label_overrides = {
    i: date.strftime('%b %d %T') for i, date in zip(dfpl.index, dfpl["Datetime"])
}

atr.line(
    x="index", 
    y="ATR", 
    color="black",
    legend_label="ATR",
    source=dfpl)


atr.yaxis[0].formatter = NumeralTickFormatter(format="0,0")


#Fin ATR

fig = column(children=[p, adx, atr2, atr], sizing_mode="scale_width")


show(fig)


#### BACKTESTING
###### Usando paquete backtesting.py

In [61]:
df_backtesting=dfprincipal.copy()
df_backtesting['isBreakOutIni'] = df_backtesting['isBreakOutIni'].shift(-1)
df_backtesting['isBreakOutFinal'] = df_backtesting['isBreakOutFinal'].shift(-1)
df_backtesting.set_index('Datetime', inplace=True)

In [62]:
class StrategyCanalBajistaAlcista(Strategy):
    def init(self):
        self.breakout_entry = self.I(lambda x: x, self.data.isBreakOutIni) #Indica la señal entrada al backtesting
        self.breakout_exit = self.I(lambda x: x, self.data.isBreakOutFinal) #Indica la señal salida al backtesting
    def next(self):
        #print(self.data)
        if self.breakout_entry[-1] == 1:
            self.position.close()
            self.buy(tag="long")  #COMPRA
        elif self.breakout_entry[-1]==-1:
            self.position.close()
            self.sell(tag="short") #VENTA
        elif self.breakout_exit[-1] == 1 and self.position:
            self.position.close() #CIERRE
        elif self.breakout_exit[-1] == -1 and self.position:
            self.position.close() #CIERRE

In [ ]:
my_stats = {}
estadisticas = pd.DataFrame()          # lista para ticker
tradesprev = pd.DataFrame()     # lista para cada caso por ticker
for ticker in df_backtesting['companyName'].unique():
    ticker_data = df_backtesting[df_backtesting['companyName'] == ticker].copy() #Filtro por cada ticker
    bt = Backtest(ticker_data, StrategyCanalBajistaAlcista, cash=10_000)
    my_stats = bt.run()
    listTrades = my_stats['_trades']
    tradesdf = pd.DataFrame(listTrades) #trades
    tradesdf['Ticker'] = ticker
    stats_filter={
        'Ticker':ticker,
        'EntryTime':my_stats['Start'],
        'ExitTime':my_stats['End'],
        'Return [%]': my_stats['Return [%]'],
        'CAGR [%]': my_stats['Return (Ann.) [%]'],
        'Sharpe Ratio': my_stats['Sharpe Ratio'],
        'Max. Drawdown [%]': my_stats['Max. Drawdown [%]'],
        'Win Rate [%]': my_stats['Win Rate [%]'],
        '# Trades': my_stats['# Trades'],
        'Expectancy [%]': my_stats['Expectancy [%]'],
        'Profit Factor': my_stats['Profit Factor'],
        'Duration': my_stats['Duration'],
        'Avg. Trade [%]':my_stats['Avg. Trade [%]'],
        'Max. Trade Duration':my_stats['Max. Trade Duration'],
        'Avg. Trade Duration':my_stats['Avg. Trade Duration']
    }
    stat=pd.DataFrame([stats_filter])
    if tradesprev.shape[0]==0:
        tradesprev = tradesdf.copy()
        estadisticas=stat.copy()
    else:
        tradesprev = pd.concat([tradesprev, tradesdf],ignore_index=True)
        estadisticas=pd.concat([estadisticas, stat],ignore_index=True)
    # Mostramos
    #print(f"Resultados para {ticker}:")
    #print(stats)   
    ##plt.title(f"Backtest -  {ticker}")
    #bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")

Backtest.run:   0%|          | 0/3289 [00:00<?, ?bar/s]

C:\Users\carlo\AppData\Local\Temp\ipykernel_2656\105393767.py:39: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")


Backtest.run:   0%|          | 0/3373 [00:00<?, ?bar/s]

C:\Users\carlo\AppData\Local\Temp\ipykernel_2656\105393767.py:39: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")


In [64]:
trades = pd.merge(df_casos, tradesprev, on = ['EntryTime','Ticker','Tag'], how='left')

In [65]:
def exitPrice(x):
    if pd.isna(x['ExitPrice_x']):
        return x['ExitPrice_y']
    else:
        return x['ExitPrice_x']
trades['ExitPrice'] = trades.apply(lambda row: exitPrice(row), axis=1)

In [66]:
trades = trades.rename(columns={'EntryPrice_x': 'EntryPrice'})
trades = trades.drop(['EntryPrice_y','ExitPrice_x', 'ExitPrice_y'], axis=1)

In [67]:
def exitTime(x):
    if pd.isna(x['ExitTime_x']):
        return x['ExitTime_y']
    else:
        return x['ExitTime_x']
trades['ExitTime'] = trades.apply(lambda row: exitTime(row), axis=1)

In [68]:
def duration(x):
    if pd.isna(x['Duration']):
        if pd.isna(x['ExitTime']):
            return pd.NaT
        else:
            return x['ExitTime']-x['EntryTime']
    else:
        return x['Duration']
trades['Duration'] = trades.apply(lambda row: duration(row), axis=1)

In [69]:
trades = trades.drop(['ExitTime_x', 'ExitTime_y'], axis=1)

In [70]:
trades=trades[['Ticker','EntryTime','ExitTime','EntryPrice','ExitPrice','Duration','Size','EntryBar','ExitBar','ReturnPct','PnL','caso','Tag']]

In [71]:
for item in estadisticas['Ticker'].unique():
    count = trades[trades["Ticker"] == item].shape[0]
    estadisticas.loc[estadisticas['Ticker'] == item, '# Trades'] = count

In [ ]:
#EXPORTACION DE DATAFRAME
path = 'data/backtesting'

# Asegurarse de que el directorio existe
os.makedirs(path, exist_ok=True)

path_trades = os.path.join(path, 'trades_cba.txt')
path_estadisticas = os.path.join(path, 'estadisticas_cba.txt')

for file_path in [path_trades, path_estadisticas]:
    if os.path.exists(file_path):
        os.remove(file_path)
    else:
        print(f"File does not exist: {file_path}. It will be created.")

trades.to_csv(path_trades, header=True, index=False, sep='\t', mode='w')
estadisticas.to_csv(path_estadisticas, header=True, index=False, sep='\t', mode='w')

In [77]:
#trades[trades['Ticker']=="SPY"].shape[0]
#estadisticas